In [ ]:
from scipy.optimize import curve_fit
import hyperspy.api as hys
import numpy as np
import matplotlib.pyplot as plt
import tkinter.filedialog as tkf
plt.rcParams['font.family'] = 'Cambria'

import matplotlib.cm as cm
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap

color_rep = ["black", "red", "green", "blue", "orange", "purple", "yellow", "lime", 
             "cyan", "magenta", "lightgray", "peru", "springgreen", "deepskyblue", 
             "hotpink", "darkgray"]
print(len(color_rep))

rgb_rep = {"black":[1,1,1,1], "red":[1,0,0,1], "green":[0,1,0,1], "blue":[0,0,1,1], "orange":[1,0.5,0,1], "purple":[1,0,1,1],
           "yellow":[1,1,0,1], "lime":[0,1,0.5,1], "cyan":[0,1,1,1]}

custom_cmap = mcolors.ListedColormap(color_rep)
bounds = np.arange(-1, len(color_rep))
norm = mcolors.BoundaryNorm(boundaries=bounds, ncolors=len(color_rep))
sm = cm.ScalarMappable(cmap=custom_cmap, norm=norm)
sm.set_array([])

cm_rep = ["gray", "Reds", "Greens", "Blues", "Oranges", "Purples"]
print(len(cm_rep))

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
spectrum_adr = []

In [ ]:
spectrum_adr.extend(tkf.askopenfilenames())
print(*spectrum_adr, sep="\n")

spectrum = hys.load(spectrum_adr[0], signal_type="EELS")
spectra = np.mean(spectrum.data, axis=1)

num_img = len(spectra)
print(num_img)

In [ ]:
si_data = []
e_ranges = []
steps = []
for i in range(num_img):
    step = spectrum.axes_manager[2].scale
    offset = spectrum.axes_manager[2].offset
    e_size = spectrum.axes_manager[2].size
    e_range = np.arange(offset, offset+e_size*step, step)
    si_data.append(spectra[i])
    e_ranges.append(e_range)
    steps.append(step)

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(1, 1, figsize=(7, 7))
for i in range(num_img):
    ax.plot(e_ranges[i], si_data[i])
fig.tight_layout()
plt.show()

In [ ]:
def gauss1(x, a, c, sigma):
    return a*np.exp(-(x-c)**2/(2*sigma**2))

def gauss2(x, a, c, sigma):
    return a*np.exp(-(x-c)**2/(2*sigma**2))

def gauss3(x, a, c, sigma):
    return a*np.exp(-(x-c)**2/(2*sigma**2))

def gauss4(x, a, c, sigma):
    return a*np.exp(-(x-c)**2/(2*sigma**2))

def gauss5(x, a, c, sigma):
    return a*np.exp(-(x-c)**2/(2*sigma**2))

def two_gauss(x, a1, a2, c1, c2, sigma1, sigma2):
    return gauss1(x, a1, c1, sigma1)+gauss2(x, a2, c2, sigma2)

def three_gauss(x, a1, a2, a3, c1, c2, c3, sigma1, sigma2, sigma3):
    return gauss1(x, a1, c1, sigma1)+gauss2(x, a2, c2, sigma2)+gauss3(x, a3, c3, sigma3)

def four_gauss(x, a1, a2, a3, a4, c1, c2, c3, c4, sigma1, sigma2, sigma3, sigma4):
    return gauss1(x, a1, c1, sigma1)+gauss2(x, a2, c2, sigma2)+gauss3(x, a3, c3, sigma3)+gauss4(x, a4, c4, sigma4)

def five_gauss(x, a1, a2, a3, a4, a5, c1, c2, c3, c4, c5, sigma1, sigma2, sigma3, sigma4, sigma5):
    return gauss1(x, a1, c1, sigma1)+gauss2(x, a2, c2, sigma2)+gauss3(x, a3, c3, sigma3)+gauss4(x, a4, c4, sigma4)+gauss5(x, a5, c5, sigma5)

In [ ]:
%matplotlib inline

In [ ]:
fit1_bound = (0.0, 1.0, 189.5, 190.5, 0.0, 1.0)
fit2_bound = (0.0, 1.0, 191.0, 192.0, 0.0, 1.0)
fit3_bound = (0.0, 1.0, 192.0, 193.0, 0.0, 1.0)
fit4_bound = (0.0, 1.0, 193.5, 194.5, 0.0, 1.0)
fit5_bound = (0.0, 1.0, 198.0, 202.0, 0.0, 5.0)

bound_left = [fit1_bound[0], fit2_bound[0], fit3_bound[0], fit4_bound[0], fit5_bound[0],
              fit1_bound[2], fit2_bound[2], fit3_bound[2], fit4_bound[2], fit5_bound[2],
              fit1_bound[4], fit2_bound[4], fit3_bound[4], fit4_bound[4], fit5_bound[4]]

bound_right = [fit1_bound[1], fit2_bound[1], fit3_bound[1], fit4_bound[1], fit5_bound[1],
              fit1_bound[3], fit2_bound[3], fit3_bound[3], fit4_bound[3], fit5_bound[3],
              fit1_bound[5], fit2_bound[5], fit3_bound[5], fit4_bound[5], fit5_bound[5]]

In [ ]:
start_ev = 185.0
end_ev = 220.0

pad = 100

fitting_result = []
for k in range(num_img):
    e_range = e_ranges[k]
    signal = si_data[k]
    start_ind = find_nearest(e_range, start_ev)
    end_ind = find_nearest(e_range, end_ev)
    print(start_ev, start_ind)
    print(end_ev, end_ind)
    
    e_range_int = e_range[start_ind:end_ind]
    signal_int = signal[start_ind:end_ind]
    signal_int = signal_int / np.max(signal_int)

    slope = (signal_int[-1] - signal_int[0]) / (e_range_int[-1] - e_range_int[0])
    intercept = signal_int[0] - slope*e_range_int[0]
    bg_line = slope*e_range_int+intercept
    
    signal_int_bg_removed = signal_int - bg_line
    signal_int_bg_removed = np.append(np.zeros(pad), signal_int_bg_removed)
    signal_int_bg_removed = np.append(signal_int_bg_removed, np.zeros(pad))
    signal_int_bg_removed = signal_int_bg_removed / np.max(signal_int_bg_removed)
    e_range_int_bg_removed = np.arange(start_ev-pad*step, end_ev+pad*step, step)

    bg_line_bg_removed = slope*e_range_int_bg_removed+intercept / np.max(signal_int_bg_removed)

    fig, ax = plt.subplots(1, 1, figsize=(7, 7))
    ax.plot(e_range_int, signal_int, 'k-')
    ax.plot(e_range_int, bg_line, 'r-')
    ax.plot(e_range_int_bg_removed, signal_int_bg_removed, 'b-')
    ax.grid()
    fig.tight_layout()
    plt.show()

    popt, pcov = curve_fit(five_gauss, e_range_int_bg_removed, signal_int_bg_removed, 
                           bounds=(bound_left, bound_right))
    print(popt)
    fitting_result.append(popt)

    fit1 = gauss1(e_range_int_bg_removed, popt[0], popt[5], popt[10])
    fit2 = gauss2(e_range_int_bg_removed, popt[1], popt[6], popt[11])
    fit3 = gauss3(e_range_int_bg_removed, popt[2], popt[7], popt[12])
    fit4 = gauss4(e_range_int_bg_removed, popt[3], popt[8], popt[13])
    fit5 = gauss5(e_range_int_bg_removed, popt[4], popt[9], popt[14])
    total = fit1 + fit2 + fit3 + fit4 + fit5

    fig, ax = plt.subplots(1, 2, figsize=(15, 7))
    ax[0].plot(e_range_int_bg_removed, signal_int_bg_removed, 'k-')
    ax[0].fill(e_range_int_bg_removed, fit1, 'r', alpha=0.3)
    ax[0].axvline(x = popt[5], color = 'r', linestyle=":")
    ax[0].text(popt[5], 0.9, '(%.3f, %.3f, %.3f)'%(popt[0], popt[5], popt[10]))
    ax[0].fill(e_range_int_bg_removed, fit2, 'b', alpha=0.3)
    ax[0].axvline(x = popt[6], color = 'r', linestyle=":")
    ax[0].text(popt[6], 0.7, '(%.3f, %.3f, %.3f)'%(popt[1], popt[6], popt[11]))
    ax[0].fill(e_range_int_bg_removed, fit3, 'g', alpha=0.3)
    ax[0].axvline(x = popt[7], color = 'r', linestyle=":")
    ax[0].text(popt[7], 0.5, '(%.3f, %.3f, %.3f)'%(popt[2], popt[7], popt[12]))
    ax[0].fill(e_range_int_bg_removed, fit4, 'p', alpha=0.3)
    ax[0].axvline(x = popt[8], color = 'r', linestyle=":")
    ax[0].text(popt[8], 0.3, '(%.3f, %.3f, %.3f)'%(popt[3], popt[8], popt[13]))
    ax[0].fill(e_range_int_bg_removed, fit5, 'p', alpha=0.3)
    ax[0].axvline(x = popt[9], color = 'r', linestyle=":")
    ax[0].text(popt[9], 0.1, '(%.3f, %.3f, %.3f)'%(popt[4], popt[9], popt[14]))
    ax[0].plot(e_range_int_bg_removed, total, 'r*', alpha=0.3)

    ax[1].plot(e_range_int_bg_removed, signal_int_bg_removed+bg_line_bg_removed, 'k-')
    ax[1].fill(e_range_int_bg_removed, fit1+bg_line_bg_removed, 'r', alpha=0.3)
    ax[1].axvline(x = popt[5], color = 'r', linestyle=":")
    ax[1].text(popt[5], 0.9, '(%.3f, %.3f, %.3f)'%(popt[0], popt[5], popt[10]))
    ax[1].fill(e_range_int_bg_removed, fit2+bg_line_bg_removed, 'b', alpha=0.3)
    ax[1].axvline(x = popt[6], color = 'r', linestyle=":")
    ax[1].text(popt[6], 0.7, '(%.3f, %.3f, %.3f)'%(popt[1], popt[6], popt[11]))
    ax[1].fill(e_range_int_bg_removed, fit3+bg_line_bg_removed, 'g', alpha=0.3)
    ax[1].axvline(x = popt[7], color = 'r', linestyle=":")
    ax[1].text(popt[7], 0.5, '(%.3f, %.3f, %.3f)'%(popt[2], popt[7], popt[12]))
    ax[1].fill(e_range_int_bg_removed, fit4+bg_line_bg_removed, 'p', alpha=0.3)
    ax[1].axvline(x = popt[8], color = 'r', linestyle=":")
    ax[1].text(popt[8], 0.3, '(%.3f, %.3f, %.3f)'%(popt[3], popt[8], popt[13]))
    ax[1].fill(e_range_int_bg_removed, fit5+bg_line_bg_removed, 'p', alpha=0.3)
    ax[1].axvline(x = popt[9], color = 'r', linestyle=":")
    ax[1].text(popt[9], 0.1, '(%.3f, %.3f, %.3f)'%(popt[4], popt[9], popt[14]))
    ax[1].plot(e_range_int_bg_removed, total+bg_line_bg_removed, 'r*', alpha=0.3)

    fig.tight_layout()
    plt.show()
print(*fitting_result, sep="\n")

In [ ]:
fit1_bound = (0.0, 1.0, 19.65, 20.75, 0.0, 3.0)
fit2_bound = (0.0, 1.0, 23.05, 24.05, 0.0, 3.0)

bound_left = [fit1_bound[0], fit2_bound[0], 
              fit1_bound[2], fit2_bound[2],
              fit1_bound[4], fit2_bound[4]]

bound_right = [fit1_bound[1], fit2_bound[1], 
              fit1_bound[3], fit2_bound[3],
              fit1_bound[5], fit2_bound[5]]

In [ ]:
start_ev = 185.0
end_ev = 220.0

pad = 100

fitting_result = []
for k in range(num_img):
    e_range = e_ranges[k]
    signal = si_data[k]
    start_ind = find_nearest(e_range, start_ev)
    end_ind = find_nearest(e_range, end_ev)
    print(start_ev, start_ind)
    print(end_ev, end_ind)
    
    e_range_int = e_range[start_ind:end_ind]
    signal_int = signal[start_ind:end_ind]
    signal_int = signal_int / np.max(signal_int)

    slope = (signal_int[-1] - signal_int[0]) / (e_range_int[-1] - e_range_int[0])
    intercept = signal_int[0] - slope*e_range_int[0]
    bg_line = slope*e_range_int+intercept
    
    signal_int_bg_removed = signal_int - bg_line
    signal_int_bg_removed = np.append(np.zeros(pad), signal_int_bg_removed)
    signal_int_bg_removed = np.append(signal_int_bg_removed, np.zeros(pad))
    signal_int_bg_removed = signal_int_bg_removed / np.max(signal_int_bg_removed)
    e_range_int_bg_removed = np.arange(start_ev-pad*step, end_ev+pad*step, step)

    bg_line_bg_removed = slope*e_range_int_bg_removed+intercept / np.max(signal_int_bg_removed)

    fig, ax = plt.subplots(1, 1, figsize=(7, 7))
    ax.plot(e_range_int, signal_int, 'k-')
    ax.plot(e_range_int, bg_line, 'r-')
    ax.plot(e_range_int_bg_removed, signal_int_bg_removed, 'b-')
    ax.grid()
    fig.tight_layout()
    plt.show()

    popt, pcov = curve_fit(two_gauss, e_range_int_bg_removed, signal_int_bg_removed, 
                           bounds=(bound_left, bound_right))
    print(popt)
    fitting_result.append(popt)

    fit1 = gauss1(e_range_int_bg_removed, popt[0], popt[2], popt[4])
    fit2 = gauss2(e_range_int_bg_removed, popt[1], popt[3], popt[5])
    total = fit1 + fit2

    fig, ax = plt.subplots(1, 2, figsize=(15, 7))
    ax[0].plot(e_range_int_bg_removed, signal_int_bg_removed, 'k-')
    ax[0].fill(e_range_int_bg_removed, fit1, 'r', alpha=0.3)
    ax[0].axvline(x = popt[2], color = 'r', linestyle=":")
    ax[0].text(popt[2], 0.5, '(%.3f, %.3f, %.3f)'%(popt[0], popt[2], popt[4]))
    ax[0].fill(e_range_int_bg_removed, fit2, 'b', alpha=0.3)
    ax[0].axvline(x = popt[3], color = 'r', linestyle=":")
    ax[0].text(popt[3], 0.5, '(%.3f, %.3f, %.3f)'%(popt[1], popt[3], popt[5]))
    ax[0].plot(e_range_int_bg_removed, total, 'r*', alpha=0.3)

    ax[1].plot(e_range_int_bg_removed, signal_int_bg_removed+bg_line_bg_removed, 'k-')
    ax[1].fill(e_range_int_bg_removed, fit1+bg_line_bg_removed, 'r', alpha=0.3)
    ax[1].axvline(x = popt[2], color = 'r', linestyle=":")
    ax[1].text(popt[2], 0.5, '(%.3f, %.3f, %.3f)'%(popt[0], popt[2], popt[4]))
    ax[1].fill(e_range_int_bg_removed, fit2+bg_line_bg_removed, 'b', alpha=0.3)
    ax[1].axvline(x = popt[3], color = 'r', linestyle=":")
    ax[1].text(popt[3], 0.5, '(%.3f, %.3f, %.3f)'%(popt[1], popt[3], popt[5]))
    ax[1].plot(e_range_int_bg_removed, total+bg_line_bg_removed, 'r*', alpha=0.3)
    fig.tight_layout()
    plt.show()

In [ ]:
fit1_bound = (0.0, 1.0, 19.65, 20.75, 0.0, 3.0)
fit2_bound = (0.0, 1.0, 23.05, 24.05, 0.0, 3.0)
fit3_bound = (0.0, 1.0, 25.6, 26.5, 0.0, 3.0)

bound_left = [fit1_bound[0], fit2_bound[0], fit3_bound[0], 
              fit1_bound[2], fit2_bound[2], fit3_bound[2],
              fit1_bound[4], fit2_bound[4], fit3_bound[4]]

bound_right = [fit1_bound[1], fit2_bound[1], fit3_bound[1], 
              fit1_bound[3], fit2_bound[3], fit3_bound[3],
              fit1_bound[5], fit2_bound[5], fit3_bound[5]]

In [ ]:
start_ev = 185.0
end_ev = 220.0

pad = 100

fitting_result = []
for k in range(num_img):
    e_range = e_ranges[k]
    signal = si_data[k]
    start_ind = find_nearest(e_range, start_ev)
    end_ind = find_nearest(e_range, end_ev)
    print(start_ev, start_ind)
    print(end_ev, end_ind)
    
    e_range_int = e_range[start_ind:end_ind]
    signal_int = signal[start_ind:end_ind]
    signal_int = signal_int / np.max(signal_int)

    slope = (signal_int[-1] - signal_int[0]) / (e_range_int[-1] - e_range_int[0])
    intercept = signal_int[0] - slope*e_range_int[0]
    bg_line = slope*e_range_int+intercept
    
    signal_int_bg_removed = signal_int - bg_line
    signal_int_bg_removed = np.append(np.zeros(pad), signal_int_bg_removed)
    signal_int_bg_removed = np.append(signal_int_bg_removed, np.zeros(pad))
    signal_int_bg_removed = signal_int_bg_removed / np.max(signal_int_bg_removed)
    e_range_int_bg_removed = np.arange(start_ev-pad*step, end_ev+pad*step, step)

    bg_line_bg_removed = slope*e_range_int_bg_removed+intercept / np.max(signal_int_bg_removed)

    fig, ax = plt.subplots(1, 1, figsize=(7, 7))
    ax.plot(e_range_int, signal_int, 'k-')
    ax.plot(e_range_int, bg_line, 'r-')
    ax.plot(e_range_int_bg_removed, signal_int_bg_removed, 'b-')
    ax.grid()
    fig.tight_layout()
    plt.show()

    popt, pcov = curve_fit(three_gauss, e_range_int_bg_removed, signal_int_bg_removed, 
                           bounds=(bound_left, bound_right))
    print(popt)
    fitting_result.append(popt)

    fit1 = gauss1(e_range_int_bg_removed, popt[0], popt[3], popt[6])
    fit2 = gauss2(e_range_int_bg_removed, popt[1], popt[4], popt[7])
    fit3 = gauss3(e_range_int_bg_removed, popt[2], popt[5], popt[8])
    total = fit1 + fit2 + fit3

    fig, ax = plt.subplots(1, 2, figsize=(15, 7))
    ax[0].plot(e_range_int_bg_removed, signal_int_bg_removed, 'k-')
    ax[0].fill(e_range_int_bg_removed, fit1, 'r', alpha=0.3)
    ax[0].axvline(x = popt[3], color = 'r', linestyle=":")
    ax[0].text(popt[3], 0.5, '(%.3f, %.3f, %.3f)'%(popt[0], popt[3], popt[6]))
    ax[0].fill(e_range_int_bg_removed, fit2, 'b', alpha=0.3)
    ax[0].axvline(x = popt[4], color = 'b', linestyle=":")
    ax[0].text(popt[4], 0.25, '(%.3f, %.3f, %.3f)'%(popt[1], popt[4], popt[7]))
    ax[0].fill(e_range_int_bg_removed, fit3, 'g', alpha=0.3)
    ax[0].axvline(x = popt[5], color = 'g', linestyle=":")
    ax[0].text(popt[5], 0.0, '(%.3f, %.3f, %.3f)'%(popt[2], popt[5], popt[8]))
    ax[0].plot(e_range_int_bg_removed, total, 'r*', alpha=0.3)

    ax[1].plot(e_range_int_bg_removed, signal_int_bg_removed+bg_line_bg_removed, 'k-')
    ax[1].fill(e_range_int_bg_removed, fit1+bg_line_bg_removed, 'r', alpha=0.3)
    ax[1].axvline(x = popt[3], color = 'r', linestyle=":")
    ax[1].text(popt[3], 0.5, '(%.3f, %.3f, %.3f)'%(popt[0], popt[3], popt[6]))
    ax[1].fill(e_range_int_bg_removed, fit2+bg_line_bg_removed, 'b', alpha=0.3)
    ax[1].axvline(x = popt[4], color = 'b', linestyle=":")
    ax[1].text(popt[4], 0.25, '(%.3f, %.3f, %.3f)'%(popt[1], popt[4], popt[7]))
    ax[1].fill(e_range_int_bg_removed, fit3+bg_line_bg_removed, 'g', alpha=0.3)
    ax[1].axvline(x = popt[5], color = 'g', linestyle=":")
    ax[1].text(popt[5], 0.0, '(%.3f, %.3f, %.3f)'%(popt[2], popt[5], popt[8]))
    ax[1].plot(e_range_int_bg_removed, total+bg_line_bg_removed, 'r*', alpha=0.3)
    fig.tight_layout()
    plt.show()

In [ ]:
fit1_bound = (0.0, 1.0, 19.65, 20.75, 0.0, 3.0)
fit2_bound = (0.0, 1.0, 23.05, 24.05, 0.0, 3.0)
fit3_bound = (0.0, 1.0, 25.6, 26.5, 0.0, 3.0)
fit4_bound = (0.0, 1.0, 27.5, 28.5, 0.0, 3.0)

bound_left = [fit1_bound[0], fit2_bound[0], fit3_bound[0], fit4_bound[0], 
              fit1_bound[2], fit2_bound[2], fit3_bound[2], fit4_bound[2],
              fit1_bound[4], fit2_bound[4], fit3_bound[4], fit4_bound[4]]

bound_right = [fit1_bound[1], fit2_bound[1], fit3_bound[1], fit4_bound[1], 
              fit1_bound[3], fit2_bound[3], fit3_bound[3], fit4_bound[3],
              fit1_bound[5], fit2_bound[5], fit3_bound[5], fit4_bound[5]]

In [ ]:
start_ev = 185.0
end_ev = 220.0

pad = 100

fitting_result = []
for k in range(num_img):
    e_range = e_ranges[k]
    signal = si_data[k]
    start_ind = find_nearest(e_range, start_ev)
    end_ind = find_nearest(e_range, end_ev)
    print(start_ev, start_ind)
    print(end_ev, end_ind)
    
    e_range_int = e_range[start_ind:end_ind]
    signal_int = signal[start_ind:end_ind]
    signal_int = signal_int / np.max(signal_int)

    slope = (signal_int[-1] - signal_int[0]) / (e_range_int[-1] - e_range_int[0])
    intercept = signal_int[0] - slope*e_range_int[0]
    bg_line = slope*e_range_int+intercept
    
    signal_int_bg_removed = signal_int - bg_line
    signal_int_bg_removed = np.append(np.zeros(pad), signal_int_bg_removed)
    signal_int_bg_removed = np.append(signal_int_bg_removed, np.zeros(pad))
    signal_int_bg_removed = signal_int_bg_removed / np.max(signal_int_bg_removed)
    e_range_int_bg_removed = np.arange(start_ev-pad*step, end_ev+pad*step, step)

    bg_line_bg_removed = slope*e_range_int_bg_removed+intercept / np.max(signal_int_bg_removed)

    fig, ax = plt.subplots(1, 1, figsize=(7, 7))
    ax.plot(e_range_int, signal_int, 'k-')
    ax.plot(e_range_int, bg_line, 'r-')
    ax.plot(e_range_int_bg_removed, signal_int_bg_removed, 'b-')
    ax.grid()
    fig.tight_layout()
    plt.show()

    popt, pcov = curve_fit(four_gauss, e_range_int_bg_removed, signal_int_bg_removed, 
                           bounds=(bound_left, bound_right))
    print(popt)
    fitting_result.append(popt)

    fit1 = gauss1(e_range_int_bg_removed, popt[0], popt[4], popt[8])
    fit2 = gauss2(e_range_int_bg_removed, popt[1], popt[5], popt[9])
    fit3 = gauss3(e_range_int_bg_removed, popt[2], popt[6], popt[10])
    fit4 = gauss4(e_range_int_bg_removed, popt[3], popt[7], popt[11])
    total = fit1 + fit2 + fit3 + fit4

    fig, ax = plt.subplots(1, 2, figsize=(15, 7))
    ax[0].plot(e_range_int_bg_removed, signal_int_bg_removed, 'k-')
    ax[0].fill(e_range_int_bg_removed, fit1, 'r', alpha=0.3)
    ax[0].axvline(x = popt[4], color = 'r', linestyle=":")
    ax[0].text(popt[4], 0.7, '(%.3f, %.3f, %.3f)'%(popt[0], popt[4], popt[8]))
    ax[0].fill(e_range_int_bg_removed, fit2, 'b', alpha=0.3)
    ax[0].axvline(x = popt[5], color = 'g', linestyle=":")
    ax[0].text(popt[5], 0.5, '(%.3f, %.3f, %.3f)'%(popt[1], popt[5], popt[9]))
    ax[0].fill(e_range_int_bg_removed, fit3, 'g', alpha=0.3)
    ax[0].axvline(x = popt[6], color = 'b', linestyle=":")
    ax[0].text(popt[6], 0.3, '(%.3f, %.3f, %.3f)'%(popt[2], popt[6], popt[10]))
    ax[0].fill(e_range_int_bg_removed, fit4, 'p', alpha=0.3)
    ax[0].axvline(x = popt[7], color = 'p', linestyle=":")
    ax[0].text(popt[7], 0.1, '(%.3f, %.3f, %.3f)'%(popt[3], popt[7], popt[11]))
    ax[0].plot(e_range_int_bg_removed, total, 'r*', alpha=0.3)

    ax[1].plot(e_range_int_bg_removed, signal_int_bg_removed+bg_line_bg_removed, 'k-')
    ax[1].fill(e_range_int_bg_removed, fit1+bg_line_bg_removed, 'r', alpha=0.3)
    ax[1].axvline(x = popt[4], color = 'r', linestyle=":")
    ax[1].text(popt[4], 0.7, '(%.3f, %.3f, %.3f)'%(popt[0], popt[4], popt[8]))
    ax[1].fill(e_range_int_bg_removed, fit2+bg_line_bg_removed, 'b', alpha=0.3)
    ax[1].axvline(x = popt[5], color = 'g', linestyle=":")
    ax[1].text(popt[5], 0.5, '(%.3f, %.3f, %.3f)'%(popt[1], popt[5], popt[9]))
    ax[1].fill(e_range_int_bg_removed, fit3+bg_line_bg_removed, 'g', alpha=0.3)
    ax[1].axvline(x = popt[6], color = 'b', linestyle=":")
    ax[1].text(popt[6], 0.3, '(%.3f, %.3f, %.3f)'%(popt[2], popt[6], popt[10]))
    ax[1].fill(e_range_int_bg_removed, fit4+bg_line_bg_removed, 'p', alpha=0.3)
    ax[1].axvline(x = popt[7], color = 'p', linestyle=":")
    ax[1].text(popt[7], 0.1, '(%.3f, %.3f, %.3f)'%(popt[3], popt[7], popt[11]))
    ax[1].plot(e_range_int_bg_removed, total+bg_line_bg_removed, 'r*', alpha=0.3)

    fig.tight_layout()
    plt.show()

In [ ]:
fit1_bound = (0.0, 1.0, 15.0, 16.0, 0.0, 1.0)
fit2_bound = (0.0, 1.0, 20.5, 21.5, 0.0, 1.0)
fit3_bound = (0.0, 1.0, 22.5, 23.5, 0.0, 1.0)
fit4_bound = (0.0, 1.0, 24.5, 25.5, 0.0, 1.0)
fit5_bound = (0.0, 1.0, 25.5, 26.5, 0.0, 5.0)
fit6_bound = (0.0, 1.0, 27.0, 29.5, 0.0, 5.0)

bound_left = [fit1_bound[0], fit2_bound[0], fit3_bound[0], fit4_bound[0], fit5_bound[0], fit6_bound[0],
              fit1_bound[2], fit2_bound[2], fit3_bound[2], fit4_bound[2], fit5_bound[2], fit6_bound[2],
              fit1_bound[4], fit2_bound[4], fit3_bound[4], fit4_bound[4], fit5_bound[4], fit6_bound[4]]

bound_right = [fit1_bound[1], fit2_bound[1], fit3_bound[1], fit4_bound[1], fit5_bound[1], fit6_bound[1],
              fit1_bound[3], fit2_bound[3], fit3_bound[3], fit4_bound[3], fit5_bound[3], fit6_bound[3],
              fit1_bound[5], fit2_bound[5], fit3_bound[5], fit4_bound[5], fit5_bound[5], fit6_bound[5]]

In [ ]:
start_ev = 10.0
end_ev = 35.0

pad = 100

fitting_result = []
fit_area = []
for k in range(num_img):
    e_range = e_ranges[k]
    signal = si_data[k]
    start_ind = find_nearest(e_range, start_ev)
    end_ind = find_nearest(e_range, end_ev)
    print(start_ev, start_ind)
    print(end_ev, end_ind)
    
    e_range_int = e_range[start_ind:end_ind]
    signal_int = signal[start_ind:end_ind]
    signal_int = signal_int / np.max(signal_int)

    slope = (signal_int[-1] - signal_int[0]) / (e_range_int[-1] - e_range_int[0])
    intercept = signal_int[0] - slope*e_range_int[0]
    bg_line = slope*e_range_int+intercept
    
    signal_int_bg_removed = signal_int - bg_line
    signal_int_bg_removed = np.append(np.zeros(pad), signal_int_bg_removed)
    signal_int_bg_removed = np.append(signal_int_bg_removed, np.zeros(pad))
    signal_int_bg_removed = signal_int_bg_removed / np.max(signal_int_bg_removed)
    e_range_int_bg_removed = np.arange(start_ev-pad*step, end_ev+pad*step, step)

    bg_line_bg_removed = slope*e_range_int_bg_removed+intercept / np.max(signal_int_bg_removed)

    fig, ax = plt.subplots(1, 1, figsize=(7, 7))
    ax.plot(e_range_int, signal_int, 'k-')
    ax.plot(e_range_int, bg_line, 'r-')
    ax.plot(e_range_int_bg_removed, signal_int_bg_removed, 'b-')
    ax.grid()
    fig.tight_layout()
    plt.show()

    popt, pcov = curve_fit(six_gauss, e_range_int_bg_removed, signal_int_bg_removed, 
                           bounds=(bound_left, bound_right))
    print(popt)
    fitting_result.append(popt)

    fit1 = gauss1(e_range_int_bg_removed, popt[0], popt[6], popt[12])
    fit2 = gauss2(e_range_int_bg_removed, popt[1], popt[7], popt[13])
    fit3 = gauss3(e_range_int_bg_removed, popt[2], popt[8], popt[14])
    fit4 = gauss4(e_range_int_bg_removed, popt[3], popt[9], popt[15])
    fit5 = gauss5(e_range_int_bg_removed, popt[4], popt[10], popt[16])
    fit6 = gauss6(e_range_int_bg_removed, popt[5], popt[11], popt[17])
    
    fit1_area = np.trapz(fit1, e_range_int_bg_removed)
    fit2_area = np.trapz(fit2, e_range_int_bg_removed)
    fit3_area = np.trapz(fit3, e_range_int_bg_removed)
    fit4_area = np.trapz(fit4, e_range_int_bg_removed)
    fit5_area = np.trapz(fit5, e_range_int_bg_removed)
    fit6_area = np.trapz(fit6, e_range_int_bg_removed)
    
    fit_area.append([fit1_area, fit2_area, fit3_area, fit4_area, fit5_area, fit6_area])
    
    total = fit1 + fit2 + fit3 + fit4 + fit5 + fit6

    fig, ax = plt.subplots(1, 2, figsize=(15, 7))
    ax[0].plot(e_range_int_bg_removed, signal_int_bg_removed, 'k-')
    ax[0].fill(e_range_int_bg_removed, fit1, 'r', alpha=0.3)
    ax[0].axvline(x = popt[5], color = 'r', linestyle=":")
    ax[0].text(popt[5], 0.9, '(%.3f, %.3f, %.3f)'%(popt[0], popt[6], popt[12]))
    ax[0].fill(e_range_int_bg_removed, fit2, 'b', alpha=0.3)
    ax[0].axvline(x = popt[6], color = 'r', linestyle=":")
    ax[0].text(popt[6], 0.7, '(%.3f, %.3f, %.3f)'%(popt[1], popt[7], popt[13]))
    ax[0].fill(e_range_int_bg_removed, fit3, 'g', alpha=0.3)
    ax[0].axvline(x = popt[7], color = 'r', linestyle=":")
    ax[0].text(popt[7], 0.5, '(%.3f, %.3f, %.3f)'%(popt[2], popt[8], popt[14]))
    ax[0].fill(e_range_int_bg_removed, fit4, 'p', alpha=0.3)
    ax[0].axvline(x = popt[8], color = 'r', linestyle=":")
    ax[0].text(popt[8], 0.3, '(%.3f, %.3f, %.3f)'%(popt[3], popt[9], popt[15]))
    ax[0].fill(e_range_int_bg_removed, fit5, 'p', alpha=0.3)
    ax[0].axvline(x = popt[9], color = 'r', linestyle=":")
    ax[0].text(popt[9], 0.1, '(%.3f, %.3f, %.3f)'%(popt[4], popt[10], popt[16]))
    ax[0].fill(e_range_int_bg_removed, fit6, 'p', alpha=0.3)
    ax[0].axvline(x = popt[9], color = 'r', linestyle=":")
    ax[0].text(popt[9], 0.1, '(%.3f, %.3f, %.3f)'%(popt[4], popt[11], popt[17]))    
    ax[0].plot(e_range_int_bg_removed, total, 'r*', alpha=0.3)

    ax[1].plot(e_range_int_bg_removed, signal_int_bg_removed+bg_line_bg_removed, 'k-')
    ax[1].fill(e_range_int_bg_removed, fit1+bg_line_bg_removed, 'r', alpha=0.3)
    ax[1].axvline(x = popt[5], color = 'r', linestyle=":")
    ax[1].text(popt[5], 0.9, '(%.3f, %.3f, %.3f)'%(popt[0], popt[6], popt[12]))
    ax[1].fill(e_range_int_bg_removed, fit2+bg_line_bg_removed, 'b', alpha=0.3)
    ax[1].axvline(x = popt[6], color = 'r', linestyle=":")
    ax[1].text(popt[6], 0.7, '(%.3f, %.3f, %.3f)'%(popt[1], popt[7], popt[13]))
    ax[1].fill(e_range_int_bg_removed, fit3+bg_line_bg_removed, 'g', alpha=0.3)
    ax[1].axvline(x = popt[7], color = 'r', linestyle=":")
    ax[1].text(popt[7], 0.5, '(%.3f, %.3f, %.3f)'%(popt[2], popt[8], popt[14]))
    ax[1].fill(e_range_int_bg_removed, fit4+bg_line_bg_removed, 'p', alpha=0.3)
    ax[1].axvline(x = popt[8], color = 'r', linestyle=":")
    ax[1].text(popt[8], 0.3, '(%.3f, %.3f, %.3f)'%(popt[3], popt[9], popt[15]))
    ax[1].fill(e_range_int_bg_removed, fit5+bg_line_bg_removed, 'p', alpha=0.3)
    ax[1].axvline(x = popt[9], color = 'r', linestyle=":")
    ax[1].text(popt[9], 0.1, '(%.3f, %.3f, %.3f)'%(popt[4], popt[10], popt[16]))
    ax[1].fill(e_range_int_bg_removed, fit6+bg_line_bg_removed, 'p', alpha=0.3)
    ax[1].axvline(x = popt[9], color = 'r', linestyle=":")
    ax[1].text(popt[9], 0.1, '(%.3f, %.3f, %.3f)'%(popt[4], popt[11], popt[17]))      
    ax[1].plot(e_range_int_bg_removed, total+bg_line_bg_removed, 'r*', alpha=0.3)

    fig.tight_layout()
    plt.show()
print(*fitting_result, sep="\n")